Another compelling example of how to compose result values is related to validation. It is taken from the [Results](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/results) page, but improved with the use of `FsToolkit.ErrorHandling`.

The Result type is typically used in **monadic** error-handling, which is often referred to as [Railway-oriented Programming](https://swlaschin.gitbooks.io/fsharpforfunandprofit/content/posts/recipe-part2.html) within the F# community.

In [ ]:
#r "nuget: FsToolkit.ErrorHandling, 4.16.0"

In [ ]:
open FsToolkit.ErrorHandling
open FsToolkit.ErrorHandling.Operator.Result

In [ ]:
let printResult result =
    match result with
    | Ok i -> printfn $"The (valid) result is: {i}"
    | Error e -> printfn $"Error: {e}"

In [ ]:
// Define a simple type which has fields that can be validated
type Request =
    { Name: string
      Email: string }

// Define some logic for what defines a valid name.
//
// Generates a Result which is an Ok if the name validates;
// otherwise, it generates a Result which is an Error.
let validateName req =
    match req.Name with
    | null -> Error "No name found."
    | "" -> Error "Name is empty."
    | "bananas" -> Error "Bananas is not a name."
    | _ -> Ok req

// Similarly, define some email validation logic.
let validateEmail req =
    match req.Email with
    | null -> Error "No email found."
    | "" -> Error "Email is empty."
    | s when s.EndsWith("bananas.com") -> Error "No email from bananas.com is allowed."
    | _ -> Ok req

let validateRequest reqResult =
    reqResult
    |> Result.bind validateName
    |> Result.bind validateEmail

let test() =
    // Now, create a Request and pattern match on the result.
    let req1 = { Name = "Phillip"; Email = "phillip@contoso.biz" }
    let res1 = validateRequest (Ok req1)
    printResult res1

    let req2 = { Name = "Phillip"; Email = "phillip@bananas.com" }
    let res2 = validateRequest (Ok req2)
    printResult res2

    let req3 = { Name = ""; Email = "phillip@bananas.com" }
    let res3 = validateRequest (Ok req3)
    printResult res3

In [ ]:
test()

Notice that for the 3rd request, even though there were 2 errors, only the first one was reported. This is an example of a short circuiting behavior. So let's see how to get all the errors at once using `FsToolkit's Validation` data type.

In [ ]:
let validateReqAllErrors req =
  validation {
    let! nameRequest = validateName req
    and! emailRequest = validateEmail req
    return req
  }

In [ ]:
let req4 = { Name = ""; Email = "phillip@bananas.com" }
let res4 = validateReqAllErrors req4
match res4 with
| Ok req -> printfn $"My request was valid! Name: {req.Name}, Email {req.Email}"  
| Error e -> printfn $"Error: {e}"